In [1]:
import cv2
import dlib
import numpy as np
import os
import pandas as pd

# Load Dlib detector and 68-point landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("models/shape_predictor_68_face_landmarks.dat/shape_predictor_68_face_landmarks.dat")

# Directories
root_input_dir = "archive_4"
root_output_dir = "Fer_processed_faces1"
os.makedirs(root_output_dir, exist_ok=True)

dataset_types = ["train", "test"]
emotions = ["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"]
scale_factor = 8
all_landmarks_data = []

print("🚀 Starting Dlib facial landmark detection with label offsets...\n")

for dataset_type in dataset_types:
    print(f"📂 Processing {dataset_type.upper()} set...")

    for emotion in emotions:
        input_path = os.path.join(root_input_dir, dataset_type, emotion)
        output_with_photo = os.path.join(root_output_dir, dataset_type, emotion, "with_photo")
        output_without_photo = os.path.join(root_output_dir, dataset_type, emotion, "without_photo")
        os.makedirs(output_with_photo, exist_ok=True)
        os.makedirs(output_without_photo, exist_ok=True)

        for filename in os.listdir(input_path):
            if not filename.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".tiff")):
                continue

            image_path = os.path.join(input_path, filename)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            if image is None:
                print(f"❌ Couldn't load image: {filename}")
                continue

            # Upscale and enhance image
            image = cv2.resize(image, (0, 0), fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_CUBIC)
            image = cv2.equalizeHist(image)
            faces = detector(image)

            if len(faces) == 0:
                print(f"⚠️ No face detected in {filename} ({emotion})")
                continue

            face = faces[0]
            landmarks = predictor(image, face)

            # Store landmark data
            row = {"dataset": dataset_type, "emotion": emotion, "filename": filename}
            for n in range(68):
                x, y = landmarks.part(n).x, landmarks.part(n).y
                row[f"landmark_{n}_x"] = x
                row[f"landmark_{n}_y"] = y
            all_landmarks_data.append(row)

            # Prepare output images
            color_image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
            blank_image = np.zeros_like(color_image)

            # Center lines on nose (point 30)
            nose_x = landmarks.part(30).x
            nose_y = landmarks.part(30).y
            h, w = image.shape
            cv2.line(color_image, (nose_x, 0), (nose_x, h), (0, 255, 0), 1)
            cv2.line(color_image, (0, nose_y), (w, nose_y), (0, 255, 0), 1)

            seen = set()
            for n in range(68):
                x = landmarks.part(n).x
                y = landmarks.part(n).y

                if (x, y) in seen:
                    continue
                seen.add((x, y))

                # --- Offset logic for better label positioning ---
                offset_x, offset_y = -6, 10
                if n in [37, 38, 39, 43, 44, 45]:  # Upper eye points
                    offset_y = -10
                elif n == 49:  # Left mouth corner
                    offset_x = -20
                    offset_y = 6
                elif n == 50:  # Top lip center
                    offset_x = 0
                    offset_y = -8
                elif n == 65:  # Chin left
                    offset_x = -14
                    offset_y = 10
                elif 48 <= n <= 54:  # Upper lip
                    offset_y = -8
                elif 55 <= n <= 59:  # Lower lip center to right
                    offset_y = 10
                elif 60 <= n <= 64:  # Inner mouth top
                    offset_y = -8
                elif 65 <= n <= 67:  # Chin right
                    offset_y = 10

                # --- Draw on image with photo ---
                cv2.circle(color_image, (x, y), 2, (0, 0, 255), -1)
                cv2.putText(color_image, str(n + 1), (x + offset_x, y + offset_y),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 0, 0), 1)

                # --- Draw on blank image ---
                cv2.circle(blank_image, (x, y), 2, (255, 255, 255), -1)
                cv2.putText(blank_image, str(n + 1), (x + offset_x, y + offset_y),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 255, 255), 1)

            # Save output images
            cv2.imwrite(os.path.join(output_with_photo, filename), color_image)
            cv2.imwrite(os.path.join(output_without_photo, filename), blank_image)

print("\n✅ All images processed successfully!")

# Save landmark CSV
csv_output = os.path.join(root_output_dir, "facial_landmarks_fer2013.csv")
pd.DataFrame(all_landmarks_data).to_csv(csv_output, index=False)
print(f"📄 Landmark coordinates saved to: {csv_output}")


🚀 Starting Dlib facial landmark detection with label offsets...

📂 Processing TRAIN set...
⚠️ No face detected in Training_10118481.jpg (angry)
⚠️ No face detected in Training_10131352.jpg (angry)
⚠️ No face detected in Training_1021836.jpg (angry)
⚠️ No face detected in Training_10333072.jpg (angry)
⚠️ No face detected in Training_10345473.jpg (angry)
⚠️ No face detected in Training_10422050.jpg (angry)
⚠️ No face detected in Training_10503476.jpg (angry)
⚠️ No face detected in Training_10539399.jpg (angry)
⚠️ No face detected in Training_10882484.jpg (angry)
⚠️ No face detected in Training_10899258.jpg (angry)
⚠️ No face detected in Training_10922970.jpg (angry)
⚠️ No face detected in Training_11023881.jpg (angry)
⚠️ No face detected in Training_11036720.jpg (angry)
⚠️ No face detected in Training_11102431.jpg (angry)
⚠️ No face detected in Training_11206889.jpg (angry)
⚠️ No face detected in Training_1120923.jpg (angry)
⚠️ No face detected in Training_11275727.jpg (angry)
⚠️ No face

KeyboardInterrupt: 

In [1]:
import cv2
import dlib
import numpy as np
import os
import pandas as pd
from mtcnn import MTCNN

# Load MTCNN and Dlib models
face_detector = MTCNN()
predictor = dlib.shape_predictor("models/shape_predictor_68_face_landmarks.dat/shape_predictor_68_face_landmarks.dat")

# Directory setup
input_root = "archive_4"
output_root = "Fer_MTCNN_processed2"
os.makedirs(output_root, exist_ok=True)

dataset_types = ["train", "test"]
emotions = ["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"]
scale_factor = 8  # Upscaling for better visibility
all_landmarks_data = []

print("🚀 Starting MTCNN + Dlib landmark detection with label offsets...\n")

for dataset_type in dataset_types:
    print(f"📂 Dataset: {dataset_type.upper()}")

    for emotion in emotions:
        input_path = os.path.join(input_root, dataset_type, emotion)
        output_with_photo = os.path.join(output_root, dataset_type, emotion, "with_photo")
        output_without_photo = os.path.join(output_root, dataset_type, emotion, "without_photo")
        os.makedirs(output_with_photo, exist_ok=True)
        os.makedirs(output_without_photo, exist_ok=True)

        for filename in os.listdir(input_path):
            if not filename.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".tiff")):
                continue

            image_path = os.path.join(input_path, filename)
            gray_img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            if gray_img is None:
                print(f"❌ Could not load: {filename}")
                continue

            # Enhance and upscale
            gray_img = cv2.resize(gray_img, (0, 0), fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_CUBIC)
            gray_img = cv2.equalizeHist(gray_img)
            rgb_img = cv2.cvtColor(gray_img, cv2.COLOR_GRAY2RGB)

            # Detect faces with MTCNN
            faces = face_detector.detect_faces(rgb_img)
            if not faces:
                print(f"⚠️ No face detected in: {filename}")
                continue

            # Get first detected face box and convert to Dlib rectangle
            x, y, w, h = faces[0]['box']
            dlib_rect = dlib.rectangle(left=x, top=y, right=x + w, bottom=y + h)

            # Extract landmarks using Dlib
            landmarks = predictor(gray_img, dlib_rect)

            # Save landmark points to CSV
            row = {"dataset": dataset_type, "emotion": emotion, "filename": filename}
            for i in range(68):
                row[f"landmark_{i}_x"] = landmarks.part(i).x
                row[f"landmark_{i}_y"] = landmarks.part(i).y
            all_landmarks_data.append(row)

            # Visualization images
            color_img = cv2.cvtColor(gray_img, cv2.COLOR_GRAY2BGR)
            blank_img = np.zeros_like(color_img)

            # Center lines at nose (point 30)
            nose_x = landmarks.part(30).x
            nose_y = landmarks.part(30).y
            h_img, w_img = gray_img.shape
            cv2.line(color_img, (nose_x, 0), (nose_x, h_img), (0, 255, 0), 1)
            cv2.line(color_img, (0, nose_y), (w_img, nose_y), (0, 255, 0), 1)

            seen = set()
            for i in range(68):
                x = landmarks.part(i).x
                y = landmarks.part(i).y

                if (x, y) in seen:
                    continue
                seen.add((x, y))

                # Offset logic for clean labels
                offset_x, offset_y = -6, 10
                if i in [37, 38, 39, 43, 44, 45]:
                    offset_y = -10
                elif i == 49:
                    offset_x = -20
                    offset_y = 6
                elif i == 50:
                    offset_x = 0
                    offset_y = -8
                elif i == 65:
                    offset_x = -14
                    offset_y = 10
                elif 48 <= i <= 54:
                    offset_y = -8
                elif 55 <= i <= 59:
                    offset_y = 10
                elif 60 <= i <= 64:
                    offset_y = -8
                elif 65 <= i <= 67:
                    offset_y = 10

                # Draw on original image
                cv2.circle(color_img, (x, y), 2, (0, 0, 255), -1)
                cv2.putText(color_img, str(i + 1), (x + offset_x, y + offset_y),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 0, 0), 1)

                # Draw on blank background
                cv2.circle(blank_img, (x, y), 2, (255, 255, 255), -1)
                cv2.putText(blank_img, str(i + 1), (x + offset_x, y + offset_y),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 255, 255), 1)

            # Save outputs
            cv2.imwrite(os.path.join(output_with_photo, filename), color_img)
            cv2.imwrite(os.path.join(output_without_photo, filename), blank_img)

# Save landmark CSV
csv_output = os.path.join(output_root, "facial_landmarks_mtcnn.csv")
pd.DataFrame(all_landmarks_data).to_csv(csv_output, index=False)

print("\n✅ All images processed and landmarks saved.")
print(f"📄 Landmark CSV saved to: {csv_output}")


C:\Users\sahug\AppData\Roaming\Python\Python39\site-packages\mtcnn\mtcnn.py:34: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


🚀 Starting MTCNN + Dlib landmark detection with label offsets...

📂 Dataset: TRAIN
1/1 [==============================] - 2s 2s/step
⚠️ No face detected in: Training_10118481.jpg
1/1 [==============================] - 0s 16ms/step
⚠️ No face detected in: Training_10131352.jpg
1/1 [==============================] - 0s 18ms/step
⚠️ No face detected in: Training_10333072.jpg
1/1 [==============================] - 0s 17ms/step
⚠️ No face detected in: Training_11102431.jpg
1/1 [==============================] - 0s 17ms/step
⚠️ No face detected in: Training_1119091.jpg
1/1 [==============================] - 0s 17ms/step
⚠️ No face detected in: Training_11330391.jpg
1/1 [==============================] - 0s 17ms/step
⚠️ No face detected in: Training_11566408.jpg
1/1 [==============================] - 0s 18ms/step
⚠️ No face detected in: Training_11571868.jpg
2/2 [==============================] - 0s 3ms/step
⚠️ No face detected in: Training_11579481.jpg
2/2 [==============================] - 

C:\Users\sahug\AppData\Roaming\Python\Python39\site-packages\mtcnn\mtcnn.py:206: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is 'Min':
C:\Users\sahug\AppData\Roaming\Python\Python39\site-packages\mtcnn\mtcnn.py:206: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is 'Min':
C:\Users\sahug\AppData\Roaming\Python\Python39\site-packages\mtcnn\mtcnn.py:206: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is 'Min':
C:\Users\sahug\AppData\Roaming\Python\Python39\site-packages\mtcnn\mtcnn.py:206: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if method is 'Min':


KeyboardInterrupt: 